In [ ]:
#Necessary Installations

%pip install beautifulsoup4
%pip install --upgrade pip
%pip install aiohttp
%pip install aiolimiter

%pip install --only-binary :all: playwright
        
!playwright install

In [9]:
import os
import string
import time
import asyncio
import random
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
from aiohttp import ClientSession
from aiolimiter import AsyncLimiter

letters = [letter for letter in string.ascii_lowercase]
letters.remove('x') #No player last names start with X

DATA_DIR = "data"
PLAYERS_DIR = os.path.join(DATA_DIR, "players")
PLAYER_STATS_DIR = os.path.join(DATA_DIR, "player_stats")

In [10]:
letter_links = [f"https://www.basketball-reference.com/players/{l}" for l in letters]

In [11]:
all_player_links = []

with open('player_links.txt', 'r') as f:
    for line in f:
        all_player_links.append(line.strip())

In [12]:
semaphore = asyncio.Semaphore(10) #10 Concurrent Tasks
rate_limiter = AsyncLimiter(15, 60) #Limit to 15 requests in a minute

In [13]:
async def get_html(url, selectors, sleep=10, retries=10):
    html = None
    
    for i in range(1, retries + 1):
        await asyncio.sleep(i * sleep + random.uniform(1, 3))
    
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                
                # Set custom timeout values
                page.set_default_navigation_timeout(150000)  # 150 seconds
                page.set_default_timeout(100000)  # 100 seconds
                
                async with rate_limiter:
                    await page.goto(url)
                    print(await page.title())

                    temp_html = []
                    for selector in selectors:
                        element = await page.query_selector(selector)
                        if element:
                            temp_html.append(await element.inner_html())

                    if temp_html:
                        html = temp_html
        except PlaywrightTimeout:
            print(f"Timeout on {url}")
            continue
        else:
            break
    
    
    return html

In [14]:
def extract_player_links_from_table(html):
    links = []
    soup = BeautifulSoup(html[0])
    rows = soup.find_all('tr')
    
    for row in rows:
        data = row.find('th')
        
        if data and len(data) > 0:
            link = data.find('a')
            if link:
                try:
                    year = int(link['href'].split("/")[-1])  # Check for year in the link as well before appending
                    if year >= 1980:
                        links.append(link['href'])
                except ValueError:
                    print(f"Skipping invalid link: {link['href']}")
                    continue
        
    return links

In [15]:
def extract_links_from_letters(html):
    links = []
    soup = BeautifulSoup(html[0])
    rows = soup.find_all('tr')
    
    for row in rows:
        data = row.find('th')
        
        if len(data) > 0:
            link = data.find('a')
            if link:
                links.append(link['href'])
        
    return links

In [16]:
async def scrape_letter(link):
    html = await get_html(link, ["#players"])
    links = extract_links_from_letters(html)
    player_links = [f"https://www.basketball-reference.com/{l}" for l in links]
    
    return player_links

In [17]:
async def scrape_player(link):
    html = await get_html(link, ["#per_game"])
    per_game_links = extract_player_links_from_table(html)
    season_links = [f"https://www.basketball-reference.com/{l}" for l in per_game_links]
    
    for url in season_links:
        save_path = os.path.join(PLAYER_STATS_DIR, "_".join(url.split("/")[-3:]))
        save_path += ".html"
        if os.path.exists(save_path):
            continue
        
        #player_season_html = await get_html(url,["#pgl_basic", "#pgl_basic_playoffs"])
        player_season_html = await get_html(url, ["#content"])
        with open(save_path, "w+") as f:
            f.write("\n".join(player_season_html))

In [27]:
async def sem_scrape_player(link):
    async with semaphore:
        await scrape_player(link)

#Use ClientSessions() to allow tasks to reuse connections and manage resources efficiently
async with ClientSession() as session:
    player_tasks = [sem_scrape_player(link) for link in all_player_links[1000:]]
    await asyncio.gather(*player_tasks)

Ed Dahler Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Mike Dabich Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Skipping invalid link: /players/d/dabicmi01/gamelog/1968/aba/
Skipping invalid link: /players/d/dabicmi01/gamelog/1968/aba/
Skipping invalid link: /players/d/dabicmi01/gamelog/1968/aba/
Howie Dallmar Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Louie Dampier Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Skipping invalid link: /players/d/dampilo01/gamelog/1968/aba/
Skipping invalid link: /players/d/dampilo01/gamelog/1969/aba/
Skipping invalid link: /players/d/dampilo01/gamelog/1970/aba/
Skipping invalid link: /players/d/dampilo01/gamelog/1971/aba/
Skipping invalid link: /players/d/dampilo01/gamelog/1972/aba/
Skipping invalid link: /players/d/dampilo01/gamelog/1973/aba/
Skipping invalid link: /players/d/dampilo01/gamelog/1974/aba/
Sk

Ben Davis 1996-97 Game Log | Basketball-Reference.com
Brad Daugherty 1990-91 Game Log | Basketball-Reference.com
Adrian Dantley 1988-89 Game Log | Basketball-Reference.com
Charles Davis Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Antonio Davis 1995-96 Game Log | Basketball-Reference.com
Antonio Daniels 2007-08 Game Log | Basketball-Reference.com
Anthony Davis 2015-16 Game Log | Basketball-Reference.com
Baron Davis 2000-01 Game Log | Basketball-Reference.com
Brad Davis 1979-80 Game Log | Basketball-Reference.com
Ben Davis 1997-98 Game Log | Basketball-Reference.com
Brian Davis 1993-94 Game Log | Basketball-Reference.com
Brad Daugherty 1991-92 Game Log | Basketball-Reference.com
Charles Davis 1981-82 Game Log | Basketball-Reference.com
Adrian Dantley 1989-90 Game Log | Basketball-Reference.com
Antonio Davis 1996-97 Game Log | Basketball-Reference.com
Antonio Daniels 2008-09 Game Log | Basketball-Reference.com
Anthony Davis 2016-17 Game Log | Basketba

Mark Davis Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Ed Davis 2018-19 Game Log | Basketball-Reference.com
Mel Davis Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Josh Davis 2004-05 Game Log | Basketball-Reference.com
Mark Davis 1988-89 Game Log | Basketball-Reference.com
Hubert Davis 1995-96 Game Log | Basketball-Reference.com
Dale Davis 1999-00 Game Log | Basketball-Reference.com
Glen Davis 2014-15 Game Log | Basketball-Reference.com
Mark Davis 1995-96 Game Log | Basketball-Reference.com
Ed Davis 2019-20 Game Log | Basketball-Reference.com
Mike Davis Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Skipping invalid link: /players/d/davismi01/gamelog/1973/aba/
Josh Davis 2005-06 Game Log | Basketball-Reference.com
Johnny Davis 1982-83 Game Log | Basketball-Reference.com
Hubert Davis 1996-97 Game Log | Basketball-Reference.com
Mike Davis Stats, Height, Weight, Position, Draft Sta

Todd Day 1992-93 Game Log | Basketball-Reference.com
Johnny Dawkins 1988-89 Game Log | Basketball-Reference.com
Tony Dawson 1994-95 Game Log | Basketball-Reference.com
Terry Davis 1997-98 Game Log | Basketball-Reference.com
Austin Daye 2009-10 Game Log | Basketball-Reference.com
Walter Davis 1984-85 Game Log | Basketball-Reference.com
Darren Daye 1983-84 Game Log | Basketball-Reference.com
Ricky Davis 2005-06 Game Log | Basketball-Reference.com
Darryl Dawkins 1983-84 Game Log | Basketball-Reference.com
Darius Days 2022-23 Game Log | Basketball-Reference.com
Johnny Dawkins 1989-90 Game Log | Basketball-Reference.com
Terry Davis 1998-99 Game Log | Basketball-Reference.com
Nando De Colo Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Todd Day 1993-94 Game Log | Basketball-Reference.com
Austin Daye 2010-11 Game Log | Basketball-Reference.com
Walter Davis 1985-86 Game Log | Basketball-Reference.com
Darryl Dawkins 1984-85 Game Log | Basketball-Reference.com


Bison Dele 1994-95 Game Log | Basketball-Reference.com
Tony Delk 1996-97 Game Log | Basketball-Reference.com
Andrew DeClercq 2002-03 Game Log | Basketball-Reference.com
Dewayne Dedmon 2018-19 Game Log | Basketball-Reference.com
DeMar DeRozan 2016-17 Game Log | Basketball-Reference.com
Carlos Delfino 2006-07 Game Log | Basketball-Reference.com
Terry Dehere 1998-99 Game Log | Basketball-Reference.com
Sam Dekker 2021-22 Game Log | Basketball-Reference.com
Matthew Dellavedova 2013-14 Game Log | Basketball-Reference.com
Vinny Del Negro 1994-95 Game Log | Basketball-Reference.com
Bison Dele 1995-96 Game Log | Basketball-Reference.com
DeMar DeRozan 2017-18 Game Log | Basketball-Reference.com
Tony Delk 1997-98 Game Log | Basketball-Reference.com
Dewayne Dedmon 2019-20 Game Log | Basketball-Reference.com
Andrew DeClercq 2003-04 Game Log | Basketball-Reference.com
Carlos Delfino 2007-08 Game Log | Basketball-Reference.com
Fennis Dembo Stats, Height, Weight, Position, Draft Status and more | Bask

...

Othell Wilson 1986-87 Game Log | Basketball-Reference.com
D.J. Wilson 2023-24 Game Log | Basketball-Reference.com
Michael Wilson 1986-87 Game Log | Basketball-Reference.com
Dylan Windler Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Kennard Winchester 1990-91 Game Log | Basketball-Reference.com
Kyle Wiltjer 2016-17 Game Log | Basketball-Reference.com
John Windsor Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Kevin Willis 1991-92 Game Log | Basketball-Reference.com
Trevor Wilson 1993-94 Game Log | Basketball-Reference.com
Lee Winfield Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Dylan Windler 2020-21 Game Log | Basketball-Reference.com
Dontonio Wingfield Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Kennard Winchester 1991-92 Game Log | Basketball-Reference.com
Corliss Williamson 2005-06 Game Log | Basketball-Reference.com
Marv Winkler Stats, H

Christian Wood Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Nate Wolters 2017-18 Game Log | Basketball-Reference.com
Randy Wittman 1988-89 Game Log | Basketball-Reference.com
David Wingate 1994-95 Game Log | Basketball-Reference.com
Al Wood 1982-83 Game Log | Basketball-Reference.com
David Wood 1988-89 Game Log | Basketball-Reference.com
Joe Wolf 1990-91 Game Log | Basketball-Reference.com
Leon Wood Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Kevin Willis 2002-03 Game Log | Basketball-Reference.com
Christian Wood 2015-16 Game Log | Basketball-Reference.com
Howard Wood 1981-82 Game Log | Basketball-Reference.com
Robert Woodard II Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Al Wood 1983-84 Game Log | Basketball-Reference.com
Randy Wittman 1989-90 Game Log | Basketball-Reference.com
David Wood 1990-91 Game Log | Basketball-Reference.com
Leon Wood 1984-85 Game Log | Basketball-R

Antoine Wright 2007-08 Game Log | Basketball-Reference.com
Mike Woodson 1986-87 Game Log | Basketball-Reference.com
Haywoode Workman 1996-97 Game Log | Basketball-Reference.com
Chris Wright Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Chris Wright 2011-12 Game Log | Basketball-Reference.com
Metta World Peace 2002-03 Game Log | Basketball-Reference.com
Delon Wright Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Orlando Woolridge 1987-88 Game Log | Basketball-Reference.com
James Worthy 1985-86 Game Log | Basketball-Reference.com
Brandan Wright 2010-11 Game Log | Basketball-Reference.com
Antoine Wright 2008-09 Game Log | Basketball-Reference.com
Mike Woodson 1987-88 Game Log | Basketball-Reference.com
Chris Wright 2012-13 Game Log | Basketball-Reference.com
Haywoode Workman 1998-99 Game Log | Basketball-Reference.com
Chris Wright 2013-14 Game Log | Basketball-Reference.com
Delon Wright 2015-16 Game Log | Basketball-Re

Tony Wroten 2013-14 Game Log | Basketball-Reference.com
George Yardley Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Metta World Peace 2012-13 Game Log | Basketball-Reference.com
Dorell Wright 2012-13 Game Log | Basketball-Reference.com
Barry Yates Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
A.J. Wynder 1990-91 Game Log | Basketball-Reference.com
Sharone Wright 1996-97 Game Log | Basketball-Reference.com
Vincent Yarbrough 2002-03 Game Log | Basketball-Reference.com
Guerschon Yabusele 2017-18 Game Log | Basketball-Reference.com
Tony Wroten 2014-15 Game Log | Basketball-Reference.com
Metta World Peace 2013-14 Game Log | Basketball-Reference.com
Wayne Yates Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Lorenzen Wright 2000-01 Game Log | Basketball-Reference.com
Charlie Yelverton Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Rich Yonakor Stats, H

Nick Young 2015-16 Game Log | Basketball-Reference.com
Stephen Zimmerman Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Derrick Zimmerman 2005-06 Game Log | Basketball-Reference.com
Thaddeus Young 2014-15 Game Log | Basketball-Reference.com
Tyler Zeller 2014-15 Game Log | Basketball-Reference.com
Danny Young 1994-95 Game Log | Basketball-Reference.com
Paul Zipser Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Wang Zhizhi 2001-02 Game Log | Basketball-Reference.com
Cody Zeller 2017-18 Game Log | Basketball-Reference.com
Nick Young 2016-17 Game Log | Basketball-Reference.com
George Zídek 1996-97 Game Log | Basketball-Reference.com
Thaddeus Young 2015-16 Game Log | Basketball-Reference.com
Ante Žižić Stats, Height, Weight, Position, Draft Status and more | Basketball-Reference.com
Stephen Zimmerman 2016-17 Game Log | Basketball-Reference.com
Tyler Zeller 2015-16 Game Log | Basketball-Reference.com
Jim Zoet Stats, Height